In [2]:

from pudu.notifications import send_change_based_notifications, detect_data_changes, NotificationService,values_are_equivalent
from pudu.apis.pudu_api import get_robot_details
import yaml
import logging
from typing import Dict, List
from datetime import datetime, timedelta
import sys
import os
# Configure logging
logger = logging.getLogger(__name__)


In [3]:
def get_robot_status(sn):
    """
    Get robot status and task information.

    Args:
        sn: Robot serial number
        get_robot_details: Function that calls the robot API

    Returns:
        dict: {
            'is_in_task': bool,
            'task_info': dict or None,
            'position': dict
        }
    """
    try:
        # Call the robot API
        #response = get_robot_details(sn)

        # Check if response is valid
        # if not response or 'data' not in response:
        #     return {
        #         'is_in_task': False,
        #         'task_info': None,
        #         'position': None
        #     }

        #
        response = \
        {
           "message":"ok",
           "data":{
               "mac":"B4:ED:D5:75:6E:EB",
                   "nickname":"出尘111",
                   "online":True,
                   "battery":89,
                   "map":{
                       "name":"4#120#地图1",
                       "lv":15,
                       "floor":""
                   },
                   "cleanbot":{
                       "rising":75,
                       "sewage":5,
                       "task":0,
                       "clean":{
                           "mode":2,
                           "report_id":"896477585377857536",
                           "msg":"",
                           "result":{
                               "time":0,
                               "area":0,
                               "status":3,
                               "break_point":{
                                   "index":-1,
                                   "vector":{
                                       "x":0,
                                       "y":0,
                                       "z":0
                                   },
                                   "clean_type":1,
                                   "start":None
                               },
                               "percentage":0,
                               "remaining_time":0,
                               "task_area":25.584999084472656,
                               "cost_water":0,
                               "cost_battery":0,
                               "charge_count":0
                           },
                           "task":{
                               "task_id":"895085098473897984",
                               "version":1693571382985,
                               "name":"yy"
                           },
                           "map":{
                               "name":"4#120#地图1",
                               "lv":6,
                               "floor":"4"
                           },

                           "config":{
                               "mode":2,
                               "vacuum_speed":2,
                               "vacuum_suction":2,
                               "wash_speed":0,
                               "wash_suction":0,
                               "wash_water":0,
                               "type":1
                           }
                       },
                       "last_mode":1,
                       "detail":"",
                       "last_task":"889606190014152704"
                   },
                   "shop":{
                       "id":2085,
                       "name":"成都办公室"
                   },
                   "position":{
                       "x":5.925793133690665,
                       "y":0.35030033129465393,
                       "z":1.7161132744962158
                   }
           }
       }
        data = response['data']

        # Extract position (always available)
        position = data.get('position', {})

        # Check if robot is in task
        cleanbot = data.get('cleanbot', {})

        clean_data = cleanbot.get('clean')

        # Robot is in task if clean data exists and is not None/empty
        is_in_task = clean_data is not None and clean_data != {}

        task_info = None
        if is_in_task:
            # Extract task information
            task_info = {
                'task_id': clean_data.get('task', {}).get('task_id'),
                'report_id': clean_data.get('report_id'),
                'map': {
                    'name': clean_data.get('map', {}).get('name'),
                    'lv': clean_data.get('map', {}).get('lv'),
                    'floor': clean_data.get('map', {}).get('floor')
                }
            }

        return {
            'is_in_task': is_in_task,
            'task_info': task_info,
            'position': position
        }

    except Exception as e:
        # Handle any errors gracefully
        print(f"Error getting robot status: {e}")
        return {
            'is_in_task': False,
            'task_info': None,
            'position': None
        }

In [4]:
get_robot_status('811064412050012')

{'is_in_task': True,
 'task_info': {'task_id': '895085098473897984',
  'report_id': '896477585377857536',
  'map': {'name': '4#120#地图1', 'lv': 6, 'floor': '4'}},
 'position': {'x': 5.925793133690665,
  'y': 0.35030033129465393,
  'z': 1.7161132744962158}}

## foxx_irvine_office and foxx_irvine_office_test have:

### pro_floor_info

floor_id       BIGINT
building_id    VARCHAR(50)
floor_number   INT
floor_name     VARCHAR(100)
floor_type     CHAR(1)
description    TEXT
floor_map      VARCHAR(255)

### mnt_robots_work_location
robot_sn      VARCHAR(100)
map_name      VARCHAR(200)
x             DOUBLE
y             DOUBLE
z             DOUBLE
status        VARCHAR(50)
update_time   DATETIME

### mnt_robot_map_floor_mapping
id            BIGINT
map_name      VARCHAR(200)
floor_id      BIGINT

## ry-vue has:

### pro_building_info 
building_id   VARCHAR(50)
project_id    VARCHAR(50)


### pro_project_info 
project_id    VARCHAR(50)
tenant_id     VARCHAR(50)
project_name  VARCHAR(255)

### pro_tenant_info 
tenant_id     VARCHAR(50)

### sys_tenant_resource
tenant_resource_id
tenant_id
resource_type (enum - robot_device, sensor_device, etc.)
resource_key (is robot_sn if resource_type is robot_device)
status (enum - 0 normal; 1 suspended)


In [5]:
# do not write pycache
import sys
sys.dont_write_bytecode = True
from pudu.app.main import *

In [6]:
config_paths = [
            'database_config.yaml',
            '../src/pudu/configs/database_config.yaml',
            'src/pudu/configs/database_config.yaml',
            'pudu/configs/database_config.yaml',
            '/opt/database_config.yaml'
]

config_path = None
for path in config_paths:
    if os.path.exists(path):
        config_path = path
        break

In [7]:
config = DatabaseConfig(config_path)
task_tables = initialize_tables_from_config(config, 'robot_task')

In [8]:
table = task_tables[0]

In [9]:
df = table.query_data_as_df()
df.head()

/Users/jiaxuchen/Foxx/Monitor/pudu_robot/src/pudu/rds/utils.py:138: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(f"SELECT * FROM {table_name}", connection)


,id,robot_sn,task_name,mode,sub_mode,type,vacuum_speed,vacuum_suction,wash_speed,wash_suction,...,duration,efficiency,remaining_time,consumption,water_consumption,progress,status,create_time,update_time,tenant_id
0,1,811064412050012,ISA 1st floor wet,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,...,93.0,358.32,2319.0,0.0,NaN,4.0,Task Cancelled,2025-03-26 03:37:12,2025-07-23 18:16:32,000000
1,2,811064412050012,ISA 1st floor wet,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,...,27.0,274.68,0.0,0.0,NaN,0.0,Task Cancelled,2025-03-26 03:37:12,2025-07-23 18:16:32,000000
2,3,811064412050012,ISA 1st floor wet,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,...,77.0,324.18,2247.0,0.0,NaN,2.0,Task Cancelled,2025-03-26 03:37:12,2025-07-23 18:16:32,000000
3,4,811064412050012,3rd floor dry,Sweeping,Carpet Vacuum,Carpet Vacuuming,Standard,Medium,Off,Off,...,1091.0,696.50,747.0,17.2,NaN,59.0,Task Cancelled,2025-03-26 03:37:12,2025-07-23 18:16:32,000000
4,5,811064412050012,4th floor dry,Sweeping,Carpet Vacuum,Carpet Vacuuming,Standard,Medium,Off,Off,...,1359.0,795.83,0.0,25.8,NaN,100.0,Task Ended,2025-03-26 03:37:12,2025-07-16 21:36:09,000000


In [10]:
orig_data = df.to_dict(orient='records')
orig_data[0]

{'id': 1,
 'robot_sn': '811064412050012',
 'task_name': 'ISA 1st floor wet',
 'mode': 'Scrubbing',
 'sub_mode': 'Custom',
 'type': 'Custom',
 'vacuum_speed': 'Off',
 'vacuum_suction': 'Off',
 'wash_speed': 'Standard',
 'wash_suction': 'Medium',
 'wash_water': 'Medium',
 'map_name': '1#11#USF-ISA-1ST-FLOORV2',
 'map_url': 'https://fr-tech-cloud-open.s3.eu-central-1.amazonaws.com/pudu_cloud_platform/map/B00C9D5916E8/76db4733adaa537fdb1db23a2594910c.png',
 'actual_area': 9.26,
 'plan_area': 229.75,
 'start_time': Timestamp('2024-09-19 21:36:33'),
 'end_time': Timestamp('2024-09-19 21:39:49'),
 'duration': 93.0,
 'efficiency': 358.32,
 'remaining_time': 2319.0,
 'consumption': 0.0,
 'water_consumption': nan,
 'progress': 4.0,
 'status': 'Task Cancelled',
 'create_time': Timestamp('2025-03-26 03:37:12'),
 'update_time': Timestamp('2025-07-23 18:16:32'),
 'tenant_id': '000000'}

In [8]:
# case 1: new record
new_data = orig_data[0]
new_data['task_name'] = 'Re-clean again'
new_data['start_time'] = datetime.now()
new_data['end_time'] = datetime.now() + timedelta(hours=1)
# new_data = pd.DataFrame([new_data])
new_data

{'id': 1,
 'robot_sn': '811064412050012',
 'task_name': 'Re-clean again',
 'mode': 'Scrubbing',
 'sub_mode': 'Custom',
 'type': 'Custom',
 'vacuum_speed': 'Off',
 'vacuum_suction': 'Off',
 'wash_speed': 'Standard',
 'wash_suction': 'Medium',
 'wash_water': 'Medium',
 'map_name': '1#11#USF-ISA-1ST-FLOORV2',
 'map_url': 'https://fr-tech-cloud-open.s3.eu-central-1.amazonaws.com/pudu_cloud_platform/map/B00C9D5916E8/76db4733adaa537fdb1db23a2594910c.png',
 'actual_area': 9.26,
 'plan_area': 229.75,
 'start_time': datetime.datetime(2025, 7, 30, 15, 35, 49, 345325),
 'end_time': datetime.datetime(2025, 7, 30, 16, 35, 49, 345376),
 'duration': 93.0,
 'efficiency': 358.32,
 'remaining_time': 2319.0,
 'consumption': 0.0,
 'water_consumption': nan,
 'progress': 4.0,
 'status': 'Task Cancelled',
 'create_time': Timestamp('2025-03-26 03:37:12'),
 'update_time': Timestamp('2025-07-23 18:16:32'),
 'tenant_id': '000000'}

In [9]:
changes = detect_data_changes(table, [new_data], table.primary_keys)
changes

{'811064412050012_Re-clean again_2025-07-30 15:35:49.345325': {'robot_id': '811064412050012',
  'primary_key_values': {'robot_sn': '811064412050012',
   'task_name': 'Re-clean again',
   'start_time': datetime.datetime(2025, 7, 30, 15, 35, 49, 345325)},
  'change_type': 'new_record',
  'changed_fields': ['id',
   'robot_sn',
   'task_name',
   'mode',
   'sub_mode',
   'type',
   'vacuum_speed',
   'vacuum_suction',
   'wash_speed',
   'wash_suction',
   'wash_water',
   'map_name',
   'map_url',
   'actual_area',
   'plan_area',
   'start_time',
   'end_time',
   'duration',
   'efficiency',
   'remaining_time',
   'consumption',
   'water_consumption',
   'progress',
   'status',
   'create_time',
   'update_time',
   'tenant_id'],
  'old_values': {},
  'new_values': {'id': 1,
   'robot_sn': '811064412050012',
   'task_name': 'Re-clean again',
   'mode': 'Scrubbing',
   'sub_mode': 'Custom',
   'type': 'Custom',
   'vacuum_speed': 'Off',
   'vacuum_suction': 'Off',
   'wash_speed': '

In [11]:
# case 2: new update
new_data = orig_data[0]
new_data['progress'] = 70

# new_data = pd.DataFrame([new_data])
new_data

{'id': 1,
 'robot_sn': '811064412050012',
 'task_name': 'ISA 1st floor wet',
 'mode': 'Scrubbing',
 'sub_mode': 'Custom',
 'type': 'Custom',
 'vacuum_speed': 'Off',
 'vacuum_suction': 'Off',
 'wash_speed': 'Standard',
 'wash_suction': 'Medium',
 'wash_water': 'Medium',
 'map_name': '1#11#USF-ISA-1ST-FLOORV2',
 'map_url': 'https://fr-tech-cloud-open.s3.eu-central-1.amazonaws.com/pudu_cloud_platform/map/B00C9D5916E8/76db4733adaa537fdb1db23a2594910c.png',
 'actual_area': 9.26,
 'plan_area': 229.75,
 'start_time': Timestamp('2024-09-19 21:36:33'),
 'end_time': Timestamp('2024-09-19 21:39:49'),
 'duration': 93.0,
 'efficiency': 358.32,
 'remaining_time': 2319.0,
 'consumption': 0.0,
 'water_consumption': nan,
 'progress': 70,
 'status': 'Task Cancelled',
 'create_time': Timestamp('2025-03-26 03:37:12'),
 'update_time': Timestamp('2025-07-23 18:16:32'),
 'tenant_id': '000000'}

In [13]:
[new_data]

[{'id': 1,
  'robot_sn': '811064412050012',
  'task_name': 'ISA 1st floor wet',
  'mode': 'Scrubbing',
  'sub_mode': 'Custom',
  'type': 'Custom',
  'vacuum_speed': 'Off',
  'vacuum_suction': 'Off',
  'wash_speed': 'Standard',
  'wash_suction': 'Medium',
  'wash_water': 'Medium',
  'map_name': '1#11#USF-ISA-1ST-FLOORV2',
  'map_url': 'https://fr-tech-cloud-open.s3.eu-central-1.amazonaws.com/pudu_cloud_platform/map/B00C9D5916E8/76db4733adaa537fdb1db23a2594910c.png',
  'actual_area': 9.26,
  'plan_area': 229.75,
  'start_time': Timestamp('2024-09-19 21:36:33'),
  'end_time': Timestamp('2024-09-19 21:39:49'),
  'duration': 93.0,
  'efficiency': 358.32,
  'remaining_time': 2319.0,
  'consumption': 0.0,
  'water_consumption': nan,
  'progress': 70,
  'status': 'Task Cancelled',
  'create_time': Timestamp('2025-03-26 03:37:12'),
  'update_time': Timestamp('2025-07-23 18:16:32'),
  'tenant_id': '000000'}]

In [12]:
changes = detect_data_changes(table, [new_data], table.primary_keys)
changes

{'811064412050012_ISA 1st floor wet_2024-09-19 21:36:33': {'robot_id': '811064412050012',
  'primary_key_values': {'robot_sn': '811064412050012',
   'task_name': 'ISA 1st floor wet',
   'start_time': Timestamp('2024-09-19 21:36:33')},
  'change_type': 'update',
  'changed_fields': ['progress'],
  'old_values': {'id': 1,
   'robot_sn': '811064412050012',
   'task_name': 'ISA 1st floor wet',
   'mode': 'Scrubbing',
   'sub_mode': 'Custom',
   'type': 'Custom',
   'vacuum_speed': 'Off',
   'vacuum_suction': 'Off',
   'wash_speed': 'Standard',
   'wash_suction': 'Medium',
   'wash_water': 'Medium',
   'map_name': '1#11#USF-ISA-1ST-FLOORV2',
   'map_url': 'https://fr-tech-cloud-open.s3.eu-central-1.amazonaws.com/pudu_cloud_platform/map/B00C9D5916E8/76db4733adaa537fdb1db23a2594910c.png',
   'actual_area': Decimal('9.26'),
   'plan_area': Decimal('229.75'),
   'start_time': datetime.datetime(2024, 9, 19, 21, 36, 33),
   'end_time': datetime.datetime(2024, 9, 19, 21, 39, 49),
   'duration': De

In [16]:
for unique_id, change_info in changes.items():
    primary_key_values = change_info.get('primary_key_values', {})

In [17]:
primary_key_values

{'robot_sn': '811064412050012',
 'task_name': 'ISA 1st floor wet',
 'start_time': Timestamp('2024-09-19 21:36:33')}

In [10]:
notification_service = NotificationService()

In [11]:
notif_success, notif_failed = send_change_based_notifications(
                    notification_service, changes, 'robot_task'
                )